In [ ]:
!pip install langchain chromadb openai tiktoken
!pip install sentence_transformers
!pip install "unstructured[pdf]"

In [ ]:
import os
import openai

os.environ["OPENAI_API_KEY"] = 'your-api-key'


In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader
from unstructured.cleaners.core import clean_extra_whitespace


loader = UnstructuredPDFLoader("/content/sber-ar-2022-ru.pdf",mode='paged', post_processors=[clean_extra_whitespace])
data = loader.load()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
from langchain.vectorstores.utils import filter_complex_metadata

data = filter_complex_metadata(data)


In [ ]:
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import Chroma

child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OpenAIEmbeddings(),persist_directory="./vector_store"
)



In [ ]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage._lc_store import create_kv_docstore
from langchain.storage import LocalFileStore


fs = LocalFileStore("./chroma_db_filestore")
store = create_kv_docstore(fs)

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

retriever.add_documents(data, ids=None)


In [ ]:
#vectorstore.persist()
#!zip -r /content/vector_store.zip /content/vector_store

In [ ]:
eval_dataset = [
    {"question": "Какая Чистая прибыль?", "page": 14 },
    {"question": "Какая прибыль на акцию?", "page": 14 },
    {"question": "Какая доля рынка в автокредитование? ", "page": 25 },
    {"question": "Количество участников сберСпасибо", "page": 29 },
    {"question": "инвнстиционный портфель Сбера?", "page": 44 },
    {"question": "Ассистент Салют MAU", "page": 56 },
    {"question": "Финансовый эффектр от применения AI", "page": 58 },
    {"question": "Сколько людей в коорпоративном управлении сбербанка и в наблюдательном совете?", "page": 89 },
    {"question": "Число сотрудников Сбера", "page": 149 },
    {"question": "Доля женщин в персонале Сбера", "page": 149 },
    {"question": "Чистая прибыль сбера за 2022", "page": 60 },
    {"question": "Какие достижения?", "page": 182 },
    {"question": "Ключевые результаты Сбербанка за 2022", "page": 222 }
]

from langchain.retrievers.multi_vector import SearchType

retriever.search_type = SearchType.similarity
retriever.search_kwargs= {"k": 6}

def evaluate_rag_system(eval_dataset):
    total_questions = len(eval_dataset)
    correct_answers = 0
    for question_data in eval_dataset:
        question = question_data["question"]
        page_number = question_data['page']
        # Предполагается, что rag_system - это функция, которая возвращает ответ и принимает вопрос и номер страницы
        ans = retriever.get_relevant_documents(question)
        # Сравнение ответа с ожидаемым
        for i in ans:
          if i.metadata['page_number'] == page_number:
            correct_answers += 1
            break
        # Вывод результата для текущего вопроса
        print(f"Вопрос: {question_data}")
        print(f"Полученный ответ: {correct_answers}")
    # Общая точность
    accuracy = (correct_answers / total_questions) * 100
    print(f"Общая точность: {accuracy}%")

In [ ]:
evaluate_rag_system(eval_dataset)
